************************************************************************************************************
* 题目：数据处理
* 制作者:版权所有,柳承睿、蒲鹏 Email:ppu@cc.ecnu.edu.cn；
* 最后更新日期：2022-6-1
* 补充掌握内容： 推荐大家访问官网   https://matplotlib.org/   以及GITHUB资源：https://github.com/search?l=Jupyter+Notebook&p=2&q=matplotlib&type=Repositories
* scipy更多示例：https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html  官方示例和JPT代码
* 声明：在转发并拷贝的过程中，请保留版权信息
************************************************************************************************************

该文件主要对数据进行处理

In [4]:
# %load "mylib\mysql_tool.py"

In [5]:
import numpy as np
from scipy.optimize import curve_fit
from mylib.mysql_tool import execute_sql
from math import e
from sklearn.linear_model import LassoCV, RidgeCV

## 实验1 获取所有repo

In [6]:
import pandas as pd
repos = execute_sql("select * from repo",False)
df_repo = pd.DataFrame(repos,columns=['repo_id','repo_name'])
df_repo.index=df_repo['repo_id'] # 将repo_id列作为DataFrame的行索引
print("repos shape:",df_repo.shape)
df_repo.to_csv("repos.csv")
df_repo.head(5)

repos shape: (1171, 2)


,repo_id,repo_name
repo_id,,
100118847,100118847,Azure/azure-docs-json-samples
100314903,100314903,Azure/autorest.common
100315083,100315083,Azure/autorest.modeler
100315457,100315457,Azure/autorest.azureresourceschema
100315623,100315623,Azure/autorest.csharp


## 实验2 获取所有的edge数据

In [7]:
edges = execute_sql("select * from edge",False)
df_edges = pd.DataFrame(edges,columns=['repo_i','repo_j','weight','date'])
print("edge shape:",df_edges.shape)
df_edges.to_csv("edges.csv")
df_edges.head(5)

edge shape: (14007, 4)


,repo_i,repo_j,weight,date
0,100118847,136426082,0,201912
1,100118847,145183271,0,201912
2,100118847,172814583,0,201912
3,100118847,173776180,0,201912
4,100118847,184655500,0,201912


## 实验3 获取想要研究的repo在某一年度的所有权重情况

### 步骤1 获取不同的repo_i列表

In [8]:
repo_i_unique=df_edges.repo_i.unique()
print("repo_i_unique shape:",repo_i_unique.shape)
print(repo_i_unique[:5])

repo_i_unique shape: (462,)
['100118847' '100314903' '100315083' '100315457' '100315623']


### 步骤2 获取这些repo的名字

In [9]:
df_repo_unique=df_repo.loc[df_repo['repo_id'].isin(repo_i_unique )] # some_values是可迭代对象
df_repo_unique.head(5)

,repo_id,repo_name
repo_id,,
100118847,100118847,Azure/azure-docs-json-samples
100314903,100314903,Azure/autorest.common
100315083,100315083,Azure/autorest.modeler
100315457,100315457,Azure/autorest.azureresourceschema
100315623,100315623,Azure/autorest.csharp


### 步骤3 查看某个仓库，在某个月的weight(包含出和入)

In [10]:
repo_id='100315083'
date='201911'

In [11]:
temp=df_edges[((df_edges.repo_j==repo_id) | (df_edges.repo_i==repo_id))&(df_edges.date==date) ]
#temp.head(5)
temp

,repo_i,repo_j,weight,date
92,100315083,100315700,7,201911
93,100315083,105314178,10,201911
94,100315083,124717436,22,201911
96,100315083,140454351,2,201911
97,100315083,165910472,10,201911
98,100315083,173348572,2,201911
99,100315083,176383947,2,201911
100,100315083,186700904,2,201911
101,100315083,24960549,41,201911
105,100315083,30773898,2,201911


### 步骤4 制作一个表格，包含了所有的要统计的仓库在2019年的weight

In [12]:
df_repo_date_weight=df_repo_unique.copy()
df_repo_date_weight=df_repo_date_weight.copy()
months_not_zeroArray = np.zeros(df_repo_date_weight.shape[0])
#df_repo_date_weight['months_zero']=0
df_repo_date_weight.head(5)

,repo_id,repo_name
repo_id,,
100118847,100118847,Azure/azure-docs-json-samples
100314903,100314903,Azure/autorest.common
100315083,100315083,Azure/autorest.modeler
100315457,100315457,Azure/autorest.azureresourceschema
100315623,100315623,Azure/autorest.csharp


该函数，将数据从mysql取出

In [13]:
def get_repo_date_weight(date_list):
    i=0;
    #df_repo_date_weight表格增加12列，用来放置每个月的度
    for date in date_list:
                df_repo_date_weight[date]="000000"
    #df_repo_date_weight表格增加1列，用来放置多少个月度没有发生变化
    for repo_id in repo_i_unique:
            months_not_zero=0
            #每个repo这个月都要累积上个月的weight
            sum_weight=0
            for date in date_list:
                #该仓库在某个月内，和其它所有仓库之间的weights
                weights=df_edges[((df_edges.repo_j==repo_id) | (df_edges.repo_i==repo_id))&(df_edges.date==date) ].weight
                if(weights.sum()>0):
                    months_not_zero+=1
                sum_weight+=weights.sum()
                df_repo_date_weight.loc[repo_id][date]=sum_weight
            #df_repo_date_weight.loc[repo_id]['months_zero']=months_zero
            months_not_zeroArray[i]=months_not_zero
            i=i+1
    df_repo_date_weight.insert(df_repo_date_weight.shape[1],'months_not_zero', months_not_zeroArray)    

In [14]:
date_list = ["201901", "201902", "201903", "201904", "201905", "201906", "201907", "201908", "201909", "201910", "201911", "201912"]
get_repo_date_weight(date_list)
df_repo_date_weight.to_csv("repo_date_weight2019.csv")

查看生成的数据表的内容

In [15]:
df_repo_date_weight

,repo_id,repo_name,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,months_not_zero
repo_id,,,,,,,,,,,,,,,
100118847,100118847,Azure/azure-docs-json-samples,0,0,0,0,0,0,0,0,0,0,0,0,0.0
100314903,100314903,Azure/autorest.common,0,0,0,0,0,0,0,0,0,0,0,0,0.0
100315083,100315083,Azure/autorest.modeler,0,0,0,0,0,0,0,0,0,0,185,185,1.0
100315457,100315457,Azure/autorest.azureresourceschema,0,0,0,0,0,0,285851,285851,287096,287096,287096,288713,3.0
100315623,100315623,Azure/autorest.csharp,0,2050,2050,3232,3232,3232,3232,3232,3232,3232,3232,8797,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87485463,87485463,Azure/api-management-policy-snippets,0,0,0,0,0,0,0,0,26,163,163,163,2.0
87970287,87970287,Azure/azure-docs-cli-python.de-de,8960,9710,9710,9710,9710,9710,9710,9710,9710,9710,9710,9710,2.0
88677612,88677612,Azure/azure-functions-vs-build-sdk,268,268,273,463,546,546,580,580,580,647,649,989,8.0


### 步骤5：观察网络随时间变化指标